In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
def load_data():
    """
    여러 CSV 파일을 로드하여 데이터를 불러옵니다.
    """
    # 재무제표 정보 로드
    comp_finance_train = pd.read_csv('재무제표정보_train.csv')
    comp_finance_test = pd.read_csv('재무제표정보_test.csv')
    comp_finance = pd.concat([comp_finance_train, comp_finance_test])
    
    # 기업 정보 로드
    comp_intro_train = pd.read_csv('기업정보요약_train.csv')
    comp_intro_test = pd.read_csv('기업정보요약_test.csv')
    comp_intro = pd.concat([comp_intro_train, comp_intro_test])
    
    # 실제 사업자번호 로드
    mapping = pd.read_csv('mapping_info.csv')
    
    comp_info_spe = pd.read_csv('기업정보상세.csv')
    
    # 산업분류코드 로드
    df_industry = pd.read_excel('한국표준산업분류표.xlsx', index_col=1).reset_index().fillna(method='ffill')
    
    # 2022년 예측값을 위한 데이터 로드
    df_predict_ex = pd.read_csv('predict_ex.csv', index_col=0)
    
    return comp_finance, comp_intro, mapping, df_industry, df_predict_ex

def preprocess_industry(df_industry):
    """
    산업분류코드를 전처리하고 필요한 칼럼만 추출합니다.
    """
    def namee(row):
        # 항목명 전처리
        return row.split('(')[0]
    
    def dot_split(row):
        # 소수점 제거
        if pd.isnull(row):
            return np.nan
        else:
            return str(row).split('.')[0]
    
    def spe_industry(row):
        # 중분류코드 전처리
        if pd.isnull(row):
            return np.nan
        elif len(str(row)) == 1:
            return '0' + str(row)
        else:
            return str(row)
    
    # 항목명과 중분류코드 전처리 적용
    df_industry['항목명'] = df_industry['항목명'].apply(namee)
    df_industry['중분류코드_str'] = df_industry['중분류코드'].apply(dot_split).apply(spe_industry)
    
    # 필요한 칼럼만 추출 및 중복 제거
    return df_industry[['대분류코드', '항목명', '중분류코드_str']].drop_duplicates()

def preprocess_comp_intro(comp_intro, df_industry_unique):
    """
    기업 정보 요약을 전처리하고 산업분류코드와 병합합니다.
    """
    def dot_split(row):
        # 소수점 제거
        if pd.isnull(row):
            return np.nan
        else:
            return str(row).split('.')[0]
    
    def spe_industry2(row):
        # 세세분류코드 전처리
        if pd.isnull(row):
            return np.nan
        elif len(str(row)) == 4:
            return '0' + str(row)
        else:
            return str(row)
    
    def indCd1(row):
        # 중분류코드 추출
        if pd.isnull(row):
            return np.nan
        else:
            return row[0:2]
    
    # 세세분류코드 전처리
    comp_intro['indCd1_str'] = comp_intro['indCd1'].apply(dot_split).apply(spe_industry2)
    comp_intro['중분류코드_str'] = comp_intro['indCd1_str'].apply(indCd1)
    
    # 중분류코드를 기준으로 기업정보에 대분류코드와 항목명을 병합
    comp_intro_m1 = pd.merge(comp_intro, df_industry_unique, on='중분류코드_str', how='left')
    comp_intro_m1 = pd.merge(comp_intro_m1,comp_info_spe, left_on='BusinessNum', right_on='사업자등록번호', how='inner')
    
    return comp_intro_m1

def preprocess_comp_intro_details(comp_intro_m1, mapping):
    """
    기업 정보와 매핑 정보를 병합하고 주소, 설립일 등을 전처리합니다.
    """
    # 마스킹된 BusinessNum을 기준으로 병합
    df = pd.merge(comp_intro_m1, mapping, left_on='BusinessNum', right_on='사업자등록번호_마스킹', how='inner')
    
    # 필요한 컬럼만 추출
    df2 = df[['BusinessNum', '설립일', '주소', '종업원수', '종업원수기준년월']]
    
    # 주소 전처리
    df2['주소1'] = df2['주소'].apply(lambda x: str(x).split(' ')[0])
    address_mapping = {
        '서울특별시': '서울', '경기도': '경기', '경상북도': '경북', '경상남도': '경남',
        '전라북도': '전북', '전라남도': '전남', '충청북도': '충북', '충청남도': '충남',
        '인천광역시': '인천', '대구광역시': '대구', '부산광역시': '부산', '대전광역시': '대전',
        '광주광역시': '광주', '울산광역시': '울산', '강원도': '강원', '세종특별자치시': '세종',
        '제주특별자치도': '제주', '제주도': '제주'
    }
    df2['주소1'] = df2['주소1'].map(address_mapping).fillna(df2['주소1'])
    
    # 설립일 전처리
    df2['설립일_dt'] = df2['설립일'].apply(lambda x: datetime.strptime(str(int(x)), '%Y%m%d') if x == x else None)
    df2['설립연도'] = df2['설립일_dt'].dt.year
    df2['연차'] = df2['설립연도'].apply(lambda x: '30년이상' if x and (2024 - x) >= 30 else (2024 - x) if x else None)
    
    # 종업원수 전처리
    df2['종업원수기준년월_dt'] = df2['종업원수기준년월'].apply(lambda x: datetime.strptime(str(int(x)), '%Y%m') if x == x else None)
    df2['종업원수기준년'] = df2['종업원수기준년월_dt'].dt.year
    
    return df2

def fill_missing_employee_counts(df2, comp_finance):
    """
    결측치가 있는 종업원수를 인건비를 통해 보완하고 이상치를 제거합니다.
    """
    # 최대 연도와 인건비 추출
    df3 = comp_finance.groupby('BusinessNum').agg({'stYear': 'max', '인건비': 'first'}).reset_index()
    df2 = df2.merge(df3, on='BusinessNum', how='left')
    
    def remove_outliers(group):
        """
        IQR 방식을 사용하여 이상치 제거
        """
        Q1 = group.quantile(0.25)
        Q3 = group.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return (group >= lower_bound) & (group <= upper_bound)
    
    def filter_outliers(group):
        """
        그룹별로 이상치 제거
        """
        return group[remove_outliers(group['종업원수']) & remove_outliers(group['인건비'])]
    
    # 이상치 제거
    df_cleaned = df2.groupby('연차').apply(filter_outliers).reset_index(drop=True)
    
    # 연차별 평균 계산
    df_mean = df_cleaned.pivot_table(index='연차', values=['인건비', '종업원수'], aggfunc='mean').reset_index()
    df_mean.columns = ['연차', '평균인건비', '평균종업원수']
    
    # 결측치 보완
    df4 = df2.merge(df_mean, on='연차', how='left')
    df4['인건비'] = df4['인건비'].fillna(df4['평균인건비'])
    df4['종업원수'] = df4['종업원수'].fillna(df4['평균종업원수'])
    
    return df4

def categorize_data(df):
    """
    연차와 주소를 범주화하여 새로운 컬럼을 생성합니다.
    """
    def convert_years(row):
        if pd.isnull(row):
            return np.nan
        elif row == '30년이상':
            return 30.0
        else:
            return float(row)
    
    def convert_years_2(row):
        if pd.isnull(row):
            return np.nan
        elif row <= 5:
            return '5년이하'
        elif row <= 10:
            return '10년이하'
        elif row <= 25:
            return '25년이하'
        elif row > 25:
            return '25년초과'
        else:
            return row
    
    # 연차를 성장단계로 범주화
    df['성장단계'] = df['연차'].apply(convert_years).apply(convert_years_2)
    
    def convert_add(row):
        if pd.isnull(row):
            return np.nan
        elif row in ['서울', '인천', '경기']:
            return '수도권'
        else:
            return '비수도권'
    
    # 주소를 수도권/비수도권으로 범주화
    df['주소'] = df['주소1'].apply(convert_add)
    
    return df

def preprocess_financial_data(comp_finance):
    """
    재무 정보를 전처리하고 결측치가 많은 칼럼을 삭제하며 이상치를 처리합니다.
    """
    # 재무 데이터를 피벗 테이블 형태로 변환
    comp_finance_final = comp_finance.pivot_table(index=['BusinessNum', 'stYear'], columns='accNm', values='acctAmt').reset_index()
    
    # 결측치 비율이 30% 이상인 열 삭제
    threshold = 0.3
    comp_finance_final_2 = comp_finance_final.loc[:, comp_finance_final.isnull().mean() < threshold]
    
    def replace_outliers_with_bounds(df, column):
        """
        IQR 방식을 사용하여 이상치를 상한값/하한값으로 치환
        """
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df.loc[df[column] > upper_bound, column] = upper_bound
        df.loc[df[column] < lower_bound, column] = lower_bound
        return df
    
    # 각 컬럼에 대해 이상치 처리
    for column in comp_finance_final_2.columns:
        comp_finance_final_2 = replace_outliers_with_bounds(comp_finance_final_2, column)
    
    return comp_finance_final_2

def train_and_predict(df_all, df_predict_ex):
    """
    모델을 학습시키고 2022년 예측값을 생성합니다.
    """
    # 불필요한 컬럼 제거
    df_all.drop(columns=['BusinessNum', 'stYear'], inplace=True)
    df_all = df_all.dropna(subset=['매출액', '영업이익', '당기순이익(손실)'])
    df_predict_ex.drop(columns=['stYear'], inplace=True)
    
    # 카테고리와 숫자형 변수 리스트 생성
    category_list = df_all.select_dtypes(include=['object']).columns
    numeric_list = df_all.select_dtypes(include=[np.number]).columns.drop(['매출액', '영업이익', '당기순이익(손실)'])
    
    # 전처리 파이프라인 정의
    numeric_pipe = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())
    category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))
    prepro_pipe = make_column_transformer((numeric_pipe, numeric_list), (category_pipe, category_list))
    
    # 학습 데이터 전처리
    X = df_all.drop(columns=['매출액', '영업이익', '당기순이익(손실)'])
    prepro_pipe.fit(X)
    X_transformed = prepro_pipe.transform(X)
    
    # 예측 데이터 전처리
    X_new = df_predict_ex.drop(columns=['BusinessNum'])
    X_new_transformed = prepro_pipe.transform(X_new)
    
    # 예측 대상 변수 목록
    targets = ['매출액', '영업이익', '당기순이익(손실)']
    predictions = df_predict_ex[['BusinessNum']].copy()
    
    for target in targets:
        Y = df_all[target]
        
        # 학습 데이터와 테스트 데이터 분리
        X_train, X_test, Y_train, Y_test = train_test_split(X_transformed, Y, random_state=42)
        
        # 모델 정의 및 학습
        model = xgb.XGBRegressor()
        model.fit(X_train, Y_train)
        
        # 예측값 저장
        predictions[target] = model.predict(X_new_transformed)
    
    # 예측 결과 저장
    predictions.to_csv('(2024_bigdata)2022_predict_1조.csv', index=False)

def prediction_2022():
    """
    전체 예측 과정을 실행하는 메인 함수
    """
    # 데이터 로드
    comp_finance, comp_intro, mapping, df_industry, df_predict_ex = load_data()
    
    # 산업분류코드 전처리
    df_industry_unique = preprocess_industry(df_industry)
    
    # 기업정보 전처리
    comp_intro_m1 = preprocess_comp_intro(comp_intro, df_industry_unique)
    
    # 세부 기업정보 전처리
    df2 = preprocess_comp_intro_details(comp_intro_m1, mapping)
    
    # 종업원수 결측치 보완 및 이상치 제거
    df4 = fill_missing_employee_counts(df2, comp_finance)
    
    # 연차와 주소 범주화
    comp_intro_m2 = categorize_data(df4)
    
    # 재무정보 전처리
    comp_finance_final = preprocess_financial_data(comp_finance)
    
    # 전처리된 데이터 병합
    df_all = pd.merge(comp_intro_m2, comp_finance_final, on='BusinessNum', how='inner')
    
    # 모델 학습 및 예측
    train_and_predict(df_all, df_predict_ex)